In [142]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import matplotlib.pyplot as plt
import urllib.request as urlreq
import statsmodels.formula.api as sm

import plotly.plotly as py
import plotly.graph_objs as go

from pandas_datareader.data import Options
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *

'''
Calculate the Black-Scholes implied volatility.

Parameters:	
price (float) – the Black-Scholes option price
S (float) – underlying asset price
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
>>> S = 100
>>> K = 100
>>> sigma = .2
>>> r = .01
>>> flag = 'c'
>>> t = .5
>>> price = black_scholes(flag, S, K, t, r, sigma)
>>> iv = implied_volatility(price, S, K, t, r, flag)
'''
%matplotlib inline

def write_excel(filename, sheetnames, df_list):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    for i, df in enumerate(df_list):
        
        df.to_excel(writer, sheet_name = sheetnames[i])

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    return

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [143]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    sp500.index.name = 'Tickers'
    return sp500

#nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#us_stocks = pd.concat([nyse,nasdaq], axis = 0).drop_duplicates()
#us_stocks = us_stocks[us_stocks['LastSale'] != 'n/a']
#us_fundamentals = get_fundas([string.replace(' ','') for string in us_stocks.index.tolist()])
us_stocks = pd.read_csv('us_stocks.csv', index_col = 0)

active_stocks = pd.read_csv('active_names.csv', index_col = 0).dropna()
active_etfs = pd.read_csv('active_etfs.csv', index_col = 0).dropna()
highest_ivs = pd.read_csv('highest_iv.csv', index_col = 0).dropna()
unusual_names = pd.read_csv('unusual_names.csv', index_col = 0).dropna()

# filtered_names = pd.read_csv('filtered_names.csv', index_col = 0).join(us_stocks, how = 'inner')[us_stocks.columns]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(str).str[:-1]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(float)
# filtered_names = filtered_names.sort_values(['Market Cap'], ascending = False)

watchlist = ['NVDA', 'FB', 'AMZN', 'NFLX', 'GOOGL', 'GOOG',
             'TSLA', 'EA', 'ATVI', 'APPL', 'MSFT', 'INTC',
             'V', 'CSCO', 'VZ', 'T', 'MA', 'ORCL', 'IBM',
             'ADBE', 'TXN', 'AVGO', 'PYPL']

In [36]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.

def yahoo_table_parse(raw_html_table):
    tickers = []
    call_times = []
    implied_7_day = []
    implied_weekly = []
    eps = []
    i = 1
    end_row = 10
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        row_items = individual_row[0].split('</span>')[:3]

        if i == 1:
            i += 1
            continue
        tick = row_items[0].split('data-symbol="')[1].split('"')[0]
        os_check = optionslam_scrape(tick)

        if type(os_check) == str:
            continue
        else:
            tickers.append(tick)
            call_times.append(row_items[0].split('data-symbol="')[1].split('"')[-1].replace('>',''))
            eps.append(row_items[1].split('</td>')[1].split('>')[1])
            implied_7_day.append(os_check['Current 7 Day Implied'].replace('%',''))
            implied_weekly.append(os_check['Implied Move Weekly'].replace('%',''))


    return pd.DataFrame({'Tickers': tickers, 'Call Times': call_times, 'EPS': eps,
                         'Current 7 Day Implied': implied_7_day,
                         'Implied Move Weekly': implied_weekly})


def yahoo_earnings(date):
    # Yahoo Earnings Calendar Check

    today = date.strftime('%Y-%m-%d')
    tables = []
    for i in range(6):
        yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
        res = requests.get(yahoo_url)
        soup = bs(requests.get(yahoo_url).text, "lxml")

        try:
            table = soup.find_all('table')[0]
            tables.append(yahoo_table_parse(table))
        except:
            print('No Table')

    return pd.concat(tables,axis = 0, ignore_index = True)

'''
Functions for pulling options data from yahoo Input is a string. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
# Function for initial querying of yahoo data
def yahoo_option_query(ticker, unix_date):
    # dt.datetime.fromtimestamp(1525996800).date()
    if unix_date == 'None':
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}'.format(ticker)
    else:
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}?date={1}'.format(ticker,str(unix_date))
        
    response = urlreq.urlopen(yahoo_query)
    data = json.loads(response.read().decode())['optionChain']['result'][0]
    
    dict_lst = []
    for key in data.keys():
        dict_lst.append(data[key])
        
    expiries = dict_lst[1]
    strikes = dict_lst[2]
    underlying = dict_lst[4]
    calls = dict_lst[5][0]['calls']
    puts = dict_lst[5][0]['puts']
    
    return (expiries, strikes, underlying, calls, puts)

# Function for creating dataframe for options contracts for a specific maturity date
def create_contract_df(option_dict_lst, strikes):
    df = pd.DataFrame(columns = ['lastPrice','volume','openInterest','bid','ask','mid','impliedVolatility','expiration'],
                      index = strikes)
    for contract in option_dict_lst:
        for col in df.columns:
            if col == 'expiration':
                df.loc[contract['strike'], col] = (dt.datetime.fromtimestamp(contract['expiration']) - dt.datetime.today()).days
            elif col == 'mid':
                df.loc[contract['strike'], col] = (contract['ask'] + contract['bid'])/2
            else:
                df.loc[contract['strike'], col] = contract[col]
    return df.dropna()

# Function for creating straddle view of options for a specific date
def option_chain(calls, puts, strikes):
    call_contracts = create_contract_df(calls, strikes)
    put_contracts = create_contract_df(puts, strikes)
    return call_contracts.join(put_contracts, how = 'inner', lsuffix='_c', rsuffix='_p')

# Function for getting full option data for a specific ticker
def pull_options(ticker):
    initial_near_contract = yahoo_option_query(ticker, 'None')
    
    expiries = initial_near_contract[0]
    options_list = [option_chain(initial_near_contract[3], initial_near_contract[4], initial_near_contract[1])]
    
    for expiry in expiries[1:]:
        next_contract = yahoo_option_query(ticker, expiry)
        options_list.append(option_chain(next_contract[3], next_contract[4], next_contract[1]))
        
    return pd.concat(options_list, axis = 0)
# ts.get_daily('AAPL')[0].tail(1)['close'][0]
'''
Function for getting all relevant earnings for a given starting week (Monday in dt.datetime(YYYY, m, d) format)
Returns a dataframe with the earnings names, implied move, price, and earnings times.
'''
def weekly_earnings_check(start_datetime, days_forward):

    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try:
            temp_earnings = yahoo_earnings(start_date)
            temp_earnings['Earnings Date'] = start_date
            temp_earnings['Last Close'] = 0
            for idx, row in temp_earnings.iterrows():
                temp_earnings.loc[idx, 'Last Close'] = ts.get_daily(row['Tickers'])[0].tail(1)['close'][0]
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    earnings_df = earnings_df[earnings_df['Last Close'] >= 30]
    earnings_df['Implied Move Weekly'] = pd.to_numeric(earnings_df['Implied Move Weekly'])
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    earnings_df['Lower Bound'] = np.round(earnings_df['Last Close']*(1 - earnings_df['Implied Move Weekly']/100),2)
    earnings_df = earnings_df.sort_values(['Earnings Date','Call Times'])
    
    return earnings_df

def stock_earnings(tick):
    yahoo_url = 'https://finance.yahoo.com/calendar/earnings/?symbol={0}'.format(tick)
    res = requests.get(yahoo_url).text
    res = res.split('table class')[1].split('</table>')[0].split('<tbody data-reactid="')[1].split('<td class="')
    earnings_cols = list(filter(lambda x: 'data-col2' in x, res))
    estimate_cols = list(filter(lambda x: 'data-col3' in x, res))
    reported_cols = list(filter(lambda x: 'data-col4' in x, res))

    earnings_days = []
    for i in earnings_cols:
        temp_date = i.split('">')[2].split('</')[0]
        temp_date = ''.join(temp_date.split(',')[:-1])
        temp_date = dt.datetime.strptime(temp_date, '%b %d %Y')
        earnings_days.append(temp_date)

    estimate_eps = []
    for i in estimate_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        estimate_eps.append(temp_est)

    reported_eps = []
    for i in reported_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        reported_eps.append(temp_est)

    earnings = pd.DataFrame({'Earnings Dates': earnings_days,
                             'EPS Estimate': estimate_eps,
                             'EPS Reported': reported_eps})
    return earnings

def old_weekly_earnings_check(start_datetime, days_forward):
    
    def yahoo_earnings_raw(raw_html_table):
        tickers = []
        eps = []
        i = 1
        end_row = 10
        for row in raw_html_table.find_all('tr'):
            # Individual row stores current row item and delimits on '\n'
            individual_row = str(row).split('\n')
            row_items = individual_row[0].split('</span>')[:3]

            if i == 1:
                i += 1
                continue
            tick = row_items[0].split('data-symbol="')[1].split('"')[0]
            tickers.append(tick)


        return pd.DataFrame({'Tickers': tickers})
    
    def yahoo_earnings_old(date):
        # Yahoo Earnings Calendar Check

        today = date.strftime('%Y-%m-%d')
        tables = []
        for i in range(6):
            yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
            res = requests.get(yahoo_url)
            soup = bs(requests.get(yahoo_url).text, "lxml")

            try:
                table = soup.find_all('table')[0]
                tables.append(yahoo_earnings_raw(table))
            except:
                print('No Table')

        return pd.concat(tables,axis = 0, ignore_index = True)
    
    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try: 
            temp_earnings = yahoo_earnings_old(start_date)
            temp_earnings['Earnings Date'] = start_date
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)
            continue

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    
    return earnings_df

    
def fundamentals(ticker):
    
    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan
        elif 'Market Cap' in row_items[0]:
            sum_item = 'Market Cap (B)'
            mkt_cap = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
            mkt_cap_amount = float(mkt_cap[:-1])
            if mkt_cap[-1] == 'M':
                sum_value = mkt_cap_amount/1000
            else:
                sum_value = mkt_cap_amount
        else:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
        
        sum_dict[sum_item] = sum_value
    
    sum_dict['Days Since Last Earnings'] = (dt.datetime.today().date() - 
                                            stock_earnings(ticker)['Earnings Dates'][1].date()).days

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        try:
            fund_lst.append(fundamentals(tick))
        except:
            continue
    return pd.concat(fund_lst,axis = 0)

# Function historical data from alpha advantage
def historical_data(ticker, day_number = 252, rolling_window = 20, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=rolling_window,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=rolling_window,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=rolling_window,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe.tail(day_number)

# Function for building a dataframe of volatilities
# Daily, Intraday, Overnight
def current_volatility(ticker_list, roll = 20):
    
    rows = []
    failed_tickers = []
    
    def failed_check(failed_lst,rows):
        if len(failed_lst) == 0:
            return failed_lst, rows
        else:
            new_lst = []
            new_rows = rows
            for tick in failed_lst:
                try: 
                    curr_vol = historical_data(tick, outsize = 'compact').tail(1)[['daily_ann','intra_ann','ovrnt_ann','close',
                                                                                   'daily_dollar_vol']]
                    curr_vol.index.name = 'Tickers'
                    curr_vol.index = [tick]
                    new_rows.append(curr_vol)
                except:
                    new_lst.append(tick)
            return failed_check(new_lst, rows)

    for tick in ticker_list:
        try: 
            curr_vol = historical_data(tick, outsize = 'compact').tail(1)[['daily_ann','intra_ann','ovrnt_ann','close',
                                                                           'daily_dollar_vol']]
            curr_vol.index.name = 'Tickers'
            curr_vol.index = [tick]
            rows.append(curr_vol)
        except:
            failed_tickers.append(tick)
            
    failed_lst, rows = failed_check(failed_tickers, rows)
        
    return pd.concat(rows, axis = 0)

# Function for pulling S&P500 data and calculating volatilities
def sp500_filter():
    sp500 = pull_sp500_list()
    sp500_vols = current_volatility(sp500.index.tolist())
    df = pd.concat([sp500_vols, sp500], axis = 1).dropna()
    
    return df

def get_raw_data(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    return data

# Function for pulling options for a given ticker
def option_filter(ticker, moneyness_thresh, dte_thresh):
    fwd_date = dt.datetime.today() + dt.timedelta(days = dte_thresh)
    tape = Options(ticker, 'yahoo')
    data = tape.get_options_data(month = fwd_date.month, year = fwd_date.year).reset_index()
    data['Moneyness'] = np.abs(data['Strike'] - data['Underlying_Price'])/data['Underlying_Price']
    
    data['DTE'] = (data['Expiry'] - dt.datetime.today()).dt.days
    data = data[['Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 'Moneyness', 'Root', 'Underlying_Price',
                 'Last','Bid','Ask']]
    data['Mid'] = data['Ask'] - data['Bid']

    filtered_data = data[(data['Moneyness'] <= moneyness_thresh) &
                         (data['DTE'] <= dte_thresh)].reset_index()[data.columns]
    put_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', 
                                                               values='IV').dropna()
    call_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', 
                                                                values='IV').dropna()
    hv_data = current_volatility([ticker])

    put_ivs['Close'] = hv_data['close'][0]
    call_ivs['Close'] = hv_data['close'][0]
    put_ivs['Daily HV'] = hv_data['daily_ann'][0]
    call_ivs['Daily HV'] = hv_data['daily_ann'][0]
    put_ivs['Intra HV'] = hv_data['intra_ann'][0]
    call_ivs['Intra HV'] = hv_data['intra_ann'][0]
    put_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    call_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    put_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]
    call_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]
    
    put_ivs['Moneyness'] = np.abs(put_ivs.index - put_ivs['Close'])/put_ivs['Close']
    call_ivs['Moneyness'] = np.abs(call_ivs.index - call_ivs['Close'])/call_ivs['Close']

    call_ivs.index.name = ticker + ' Call Strike'
    put_ivs.index.name = ticker + ' Put Strike'
    return call_ivs, put_ivs

def phase1_options_filter(ticker_lst, hv_day_roll, atm_distance, dte_thresh, iv_hv_thresh):
    
    pulled_options = []
    calls = {}
    puts = {}

    moneyness_thresh = atm_distance

    for tick in ticker_lst:
        try:
            call, put = option_filter(tick, moneyness_thresh, dte_thresh)
            
            call_check = call.sort_values('Moneyness').head(1)
            # Checking IV vs HV of the options chain out more than 2 weeks
            iv_chain_to_check = list(filter(lambda x: x >= hv_day_roll, 
                                            list(filter(lambda x: type(x) == int,call_check.columns.tolist()))))[0]
            
            average_difference_iv_hv = abs(call_check[iv_chain_to_check] - call_check['Intra HV']).mean()

            if average_difference_iv_hv >= iv_hv_thresh:
                calls[tick] = call
                puts[tick] = put
                pulled_options.append(tick)
        except:
            pass
        
    return pulled_options, calls, puts

def greek_calc(ticker, dte_ub, dte_lb, prem_price_use = 'Mid', delta_filter = 0.2, expiry_set = 0):
    options_chain = get_raw_data(ticker).reset_index()
    options_chain = options_chain[['Strike','Expiry','Type','Last','Bid','Ask','Vol','Open_Int','IV','Underlying_Price']]
    df = options_chain
    df['DTE'] = (df['Expiry'] - dt.datetime.today()).dt.days
    df['Mid'] = (df['Ask'] + df['Bid'])/2
    df = df[(df['DTE'] <= dte_ub) & (df['DTE'] >= dte_lb)]
    df = df.reset_index()[df.columns]
    
    year = 365
    premiums = df[prem_price_use].values # 'Last' or 'Mid'
    strikes = df['Strike'].values
    time_to_expirations = df['DTE'].values
    ivs = df['IV'].values
    underlying = df['Underlying_Price'].values[0]
    types = df['Type'].values

    # Make sure nothing thows up
    assert len(premiums) == len(strikes)
    assert len(strikes) == len(time_to_expirations)

    sigmas = []
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    prices = []
    for premium, strike, time_to_expiration, flag in zip(premiums, strikes, time_to_expirations, types):

        # Constants
        P = premium
        S = underlying
        K = strike
        t = time_to_expiration/float(year)
        r = 0.005 / 100
        q = 0 / 100
        try:
            sigma = implied_volatility(P, S, K, t, r, q, flag[0])
            sigmas.append(sigma)
        except:
            sigma = 0.0
            sigmas.append(sigma)

        try:
            delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
            deltas.append(delta)
        except:
            delta = 0.0
            deltas.append(delta)

        try:
            gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
            gammas.append(gamma)
        except:
            gamma = 0.0
            gammas.append(gamma)

        try:
            theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
            thetas.append(theta)
        except:
            theta = 0.0
            thetas.append(theta)

        try:
            vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
            vegas.append(vega)
        except:
            vega = 0.0
            vegas.append(vega)

    ivs = np.array(sigmas)
    df['Calc IV'] = ivs
    df['Delta'] = deltas
    df['Gamma'] = gammas
    df['Theta'] = thetas
    df['Vega'] = vegas
    df = df.dropna()

    expiry_filter = df.sort_values('DTE')['DTE'].drop_duplicates().values[min(expiry_set, 
                                                                              len(df.sort_values('DTE')['DTE'].drop_duplicates()))]

    calls = df[(abs(df['Delta']) >= delta_filter) & 
               (df['Type'] == 'call') & 
               (df['DTE'] == expiry_filter)].reset_index()[df.columns]
    puts = df[(abs(df['Delta']) >= delta_filter) & 
              (df['Type'] == 'put') & 
              (df['DTE'] == expiry_filter)].reset_index()[df.columns]
    
    return calls, puts

def price_sim(options_df, price_change, vol_change, days_change, iv_tag = 'Calc IV', output = 'All'):
    '''
    output types can be: All, Price, Delta, Gamma, Vega, Theta
    '''
    year = 365
    strikes = options_df['Strike'].values
    time_to_expirations = options_df['DTE'].values
    ivs = options_df[iv_tag].values
    underlying = options_df['Underlying_Price'].values[0]
    types = options_df['Type'].values

    # Tweaking changes
    prices = []
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    for sigma, strike, time_to_expiration, flag in zip(ivs, strikes, time_to_expirations, types):

        # Constants
        S = underlying*(1 + price_change)
        t = max(time_to_expiration - days_change, 0)/float(year)
        sigma = sigma + vol_change
        K = strike
        r = 0.005 / 100
        q = 0 / 100
        
        if (output == 'All') or (output == 'Price'):
            if days_change == time_to_expiration:
                if flag == 'call':
                    price = max(S - K, 0.0)
                else:
                    price = max(K - S, 0.0)
                prices.append(price)
            else:
                try:
                    price = py_vollib.black_scholes_merton.black_scholes_merton(flag[0], S, K, t, r, sigma, q)
                    prices.append(price)
                except:
                    price = 0.0
                    prices.append(price)
                    
        if (output == 'All') or (output == 'Delta'):
            try:
                delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
                deltas.append(delta)
            except:
                delta = 0.0
                deltas.append(delta)
        
        if (output == 'All') or (output == 'Gamma'):
            try:
                gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
                gammas.append(gamma)
            except:
                gamma = 0.0
                gammas.append(gamma)
            
        if (output == 'All') or (output == 'Theta'):
            try:
                theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
                thetas.append(theta)
            except:
                theta = 0.0
                thetas.append(theta)
        
        if (output == 'All') or (output == 'Vega'):
            try:
                vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
                vegas.append(vega)
            except:
                vega = 0.0
                vegas.append(vega)
            
    df = options_df[['Strike','Expiry','DTE','Type','Last','Bid','Ask','Mid','Underlying_Price']]
    if (output == 'All') or (output == 'Price'):
        df['Simulated Price'] = prices
        df['Price Change'] = df['Simulated Price']/df['Mid'] - 1
    if (output == 'All') or (output == 'Delta'):
        df['Delta'] = deltas
    if (output == 'All') or (output == 'Gamma'):
        df['Gamma'] = gammas
    if (output == 'All') or (output == 'Theta'):
        df['Theta'] = thetas
    if (output == 'All') or (output == 'Vega'):
        df['Vega'] = vegas
    df = df.dropna()
    return df

def position_sim(position_df, holdings, price_change, vol_change, dte_change, iv_tag = 'Calc IV', output = 'All'):
    position = position_df
    position['Cost'] = position['Mid']
    position['Pos'] = holdings
    position_dict = {}
    position_dict['Total Cost'] = sum(position['Cost']*position['Pos'])
    position_dict['Original Delta'] = sum(position['Delta']*position['Pos'])
    position_dict['Original Gamma'] = sum(position['Gamma']*position['Pos'])
    position_dict['Original Theta'] = sum(position['Theta']*position['Pos'])
    position_dict['Original Vega'] = sum(position['Vega']*position['Pos'])
    
    if (output == 'PnL') or (output == 'Percent Return'):
        simulation = price_sim(position, price_change, vol_change, dte_change, iv_tag, 'Price')
    else:
        simulation = price_sim(position, price_change, vol_change, dte_change, iv_tag, output)
    
    if (output == 'All') or (output == 'PnL') or (output == 'Percent Return'):
        position_dict['Simulated Price'] = sum(simulation['Simulated Price']*position['Pos'])
        position_dict['PnL'] = sum(simulation['Simulated Price']*position['Pos']) - position_dict['Total Cost']
        if position_dict['Total Cost'] > 0:
            position_dict['Percent Return'] = position_dict['PnL']/position_dict['Total Cost']
        else:
            position_dict['Percent Return'] = -position_dict['PnL']/position_dict['Total Cost']
            
    if (output == 'All') or (output == 'Delta'):
        position_dict['Simulated Delta'] = sum(simulation['Delta']*position['Pos'])
        
    if (output == 'All') or (output == 'Gamma'):
        position_dict['Simulated Gamma'] = sum(simulation['Gamma']*position['Pos'])
        
    if (output == 'All') or (output == 'Theta'):
        position_dict['Simulated Theta'] = sum(simulation['Theta']*position['Pos'])
        
    if (output == 'All') or (output == 'Vega'):
        position_dict['Simulated Vega'] = sum(simulation['Vega']*position['Pos'])
    
    outframe = pd.DataFrame(position_dict, index = [vol_change])
    return outframe

def all_options(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data().reset_index()
    
    data['Moneyness'] = np.abs(data['Strike'] - data['Underlying_Price'])/data['Underlying_Price']
    
    data['DTE'] = (data['Expiry'] - dt.datetime.today()).dt.days
    data = data[['Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 'Moneyness', 'Root', 'Underlying_Price',
                 'Last','Bid','Ask']]
    data['Mid'] = data['Ask'] - data['Bid']
    
    year = 365
    strikes = data['Strike'].values
    time_to_expirations = data['DTE'].values
    ivs = data['IV'].values
    underlying = data['Underlying_Price'].values[0]
    types = data['Type'].values

    # Make sure nothing thows up
    assert len(strikes) == len(time_to_expirations)

    sigmas = data['IV']
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    for sigma, strike, time_to_expiration, flag in zip(sigmas, strikes, time_to_expirations, types):

        # Constants
        S = underlying
        K = strike
        t = time_to_expiration/float(year)
        r = 0.005 / 100
        q = 0 / 100

        try:
            delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
            deltas.append(delta)
        except:
            delta = 0.0
            deltas.append(delta)

        try:
            gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
            gammas.append(gamma)
        except:
            gamma = 0.0
            gammas.append(gamma)

        try:
            theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
            thetas.append(theta)
        except:
            theta = 0.0
            thetas.append(theta)

        try:
            vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
            vegas.append(vega)
        except:
            vega = 0.0
            vegas.append(vega)

    data['Delta'] = deltas
    data['Gamma'] = gammas
    data['Theta'] = thetas
    data['Vega'] = vegas

    return data.dropna().reset_index()[df.columns]

def combo_check(df, dte_spec, type_spec, short_pos = 1, long_pos = 2, order_style = 'market'):
    opt_df = df[(df['DTE'] == dte_spec) & 
                (df['Type'] == type_spec) &
                (abs(df['Delta']) <= 0.5)].reset_index()[df.columns]

    short_strike = []
    long_strike = []
    net_credit = []
    max_loss = []
    pos_delta = []
    pos_theta = []
    pos_gamma = []
    pos_vega = []
    short_bids =[]
    short_asks = []
    long_bids = []
    long_asks = []

    for i, low_strike_col in opt_df.iterrows():
        if i == (len(opt_df) - 1):
            break
        else:
            for j, high_strike_col in opt_df.iloc[i:,].iterrows():
                if low_strike_col.Type == 'call':
                    curr_short = low_strike_col
                    curr_long = high_strike_col
                else:
                    curr_short = high_strike_col
                    curr_long = low_strike_col
                
                curr_max_loss = curr_long.Strike - curr_short.Strike

                if order_style == 'market':
                    curr_credit = short_pos*curr_short.Bid - long_pos*curr_long.Ask
                elif order_style == 'mid':
                    curr_credit == short_pos*curr_short.Mid - long_pos*curr_long.Mid
                else:
                    curr_credit == short_pos*curr_short.Last - long_pos*curr_long.Last

                if curr_credit >= 0:
                    short_strike.append(curr_short.Strike)
                    long_strike.append(curr_long.Strike)
                    pos_delta.append(long_pos*curr_long.Delta - short_pos*curr_short.Delta)
                    pos_theta.append(long_pos*curr_long.Theta - short_pos*curr_short.Theta)
                    pos_gamma.append(long_pos*curr_long.Gamma - short_pos*curr_short.Gamma)
                    pos_vega.append(long_pos*curr_long.Vega - short_pos*curr_short.Vega)
                    net_credit.append(curr_credit)
                    max_loss.append(curr_max_loss + curr_credit)
                    short_bids.append(curr_short.Bid)
                    short_asks.append(curr_short.Ask)
                    long_bids.append(curr_long.Bid)
                    long_asks.append(curr_long.Ask)

    comb_df = pd.DataFrame({'Short Strike': short_strike, 'Long Strike': long_strike,
                            'Net Credit': net_credit, 'Max Loss': max_loss, 'Delta': pos_delta,
                            'Theta': pos_theta, 'Gamma': pos_gamma, 'Vega': pos_vega,
                            'Short Bid': short_bids, 'Short Ask': short_asks, 
                            'Long Bid': long_bids, 'Long Ask': long_asks},
                           index = range(len(net_credit)))
    comb_df['DTE'] = dte_spec
    comb_df['Type'] = type_spec
    return comb_df.reset_index()[comb_df.columns]

def combo_scanner(df, short_pos, long_pos, order_style, theta_thresh, delta_thresh,
                  dte_lb, dte_ub):

    dtes = df.DTE.drop_duplicates().values
    types = ['call','put']
    short_pos = 1
    long_pos = 2
    order_style = 'market'

    all_combos = []
    for type_spec in types:
        for dte_spec in dtes:
            all_combos.append(combo_check(df, dte_spec, type_spec, short_pos = 1, long_pos = 2, order_style = 'market'))

    all_combo_df = pd.concat(all_combos, axis = 0)
    return all_combo_df[(all_combo_df['DTE'] >= dte_lb) &
                        (all_combo_df['DTE'] <= dte_ub) &
                        (all_combo_df['Theta'] >= theta_thresh) &
                        (abs(all_combo_df['Delta']) <= delta_thresh)].sort_values(['Max Loss'],
                                                                                  ascending = False).reset_index()[all_combo_df.columns]

def back_ratio_opt(ticker_lst, max_rows, short_pos, long_pos, order_style, theta_thresh, delta_thresh,
                   dte_lb, dte_ub):
    
    combo_lst = []
    failed_names = []
    for ticker in ticker_lst:
        try: 
            curr_df = combo_scanner(all_options(ticker), short_pos, long_pos, order_style, theta_thresh, 
                                    delta_thresh, dte_lb, dte_ub).head(max_rows)
            curr_df['Ticker'] = ticker
            combo_lst.append(curr_df)
        except:
            failed_names.append(ticker)
    
    out_df = pd.concat(combo_lst, axis = 0)
    
    return out_df.sort_values(['Max Loss'], ascending = False).reset_index()[out_df.columns]

In [10]:
start_time = time.time()

# active_stock_fundas = get_fundas(active_stocks[active_stocks['Last'] >= 30].index.tolist())
# active_stock_fundas = active_stock_fundas[active_stock_fundas['Market Cap (B)'] >= 5]
# tick_lst = july16_earnings['Tickers'].tolist() #
tick_lst = active_stocks[active_stocks['Last'] >= 30].index.tolist()
etfs_lst = active_etfs[active_etfs['Last'] >= 50].index.tolist()
days_since_earnings_threshold = 30
mkt_cap_thresh = 5
moneyness_thresh = 0.05
hv_roll = 22
dte_thresh = 30
sort_on = 'intra_ann'
max_options_num = 10 # Maximum number of names to look up
iv_hv_thres = 0.9

vols = current_volatility(tick_lst, roll = 22)
etfs = current_volatility(etfs_lst, roll = 22).sort_values([sort_on], ascending = False).dropna()
vols = vols.sort_values([sort_on], ascending = False).dropna()
fundas = get_fundas(vols.index.tolist())
tickers = pd.concat([vols,fundas],axis = 1)
# tickers = tickers[(tickers['Days Since Last Earnings'] >= days_since_earnings_threshold) &
#                   (tickers['Market Cap (B)'] <= 1)].sort_values([sort_on], ascending = False)

# etf_ticks = etfs.sort_values([sort_on], ascending = False)

# pulled_options, pulled_calls, pulled_puts = phase1_options_filter(tickers.index.tolist()[:max_options_num], 
#                                                                   hv_roll, moneyness_thresh, dte_thresh,
#                                                                   iv_hv_thres)

# pulled_etfs, pulled_etfcalls, pulled_etfputs = phase1_options_filter(etf_ticks.index.tolist()[:max_options_num], 
#                                                                      hv_roll, moneyness_thresh, dte_thresh,
#                                                                      iv_hv_thres)

# filtered_single_names = (tickers.transpose()[pulled_options]).transpose()
# filtered_single_names = (tickers.transpose()[pulled_options]).transpose()
# filtered_single_names['IV'] = np.nan
# for stock in pulled_options:
#     filtered_single_names.loc[stock,'IV'] = pulled_puts[stock].sort_values(['Moneyness']).head(1).iloc[:, 0:-6].iloc[:,-1].iloc[0]
    
# filtered_etfs = (etf_ticks.transpose()[pulled_etfs]).transpose()
# filtered_etfs = (etf_ticks.transpose()[pulled_etfs]).transpose()
# filtered_etfs['IV'] = np.nan
# for etf in pulled_etfs:
#     filtered_etfs.loc[etf,'IV'] = pulled_etfputs[etf].sort_values(['Moneyness']).head(1).iloc[:, 0:-6].iloc[:,-1].iloc[0]
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 38761.15459251404 seconds ---


In [108]:
tickers

,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,1y Target Est,Beta,Days Since Last Earnings,Div,EPS (TTM),Earnings Date,Ex-Dividend Date,Market Cap (B),PE Ratio (TTM),Yield
A,0.200783,0.276692,0.106995,62.790,0.803410,77.50,1.48,38.0,0.60,0.73,,2018-07-02,20.090000,86.13,0.90
AA,0.343577,0.566645,0.211666,45.270,0.962479,66.00,N/A,63.0,NaN,0.76,,N/A,8.441000,59.64,NaN
AABA,0.167111,0.283243,0.155972,81.110,0.843951,88.00,2.02,59.0,NaN,29.96,,N/A,64.929000,2.71,NaN
AAL,0.258210,0.437851,0.113163,42.030,0.675188,57.59,1.31,56.0,0.40,3.65,,2018-05-07,19.676000,11.53,0.93
AAOI,0.786891,1.326712,0.250431,42.150,2.086875,37.29,1.35,119.0,NaN,2.79,,N/A,0.825255,15.10,NaN
AAP,0.273569,0.504520,0.082653,139.160,2.383354,125.50,0.94,30.0,0.24,6.80,,2018-06-21,10.302000,20.46,0.17
AAPL,0.121974,0.153121,0.117578,186.500,1.426774,199.68,1.15,51.0,2.92,10.34,,2018-05-11,918.787000,18.08,1.57
AAXN,0.396124,0.582264,0.191753,68.450,1.635451,52.33,0.96,113.0,NaN,0.25,N/A,N/A,3.935000,271.63,NaN
ABBV,0.254429,0.276730,0.162155,98.250,1.570857,113.86,1.63,56.0,3.84,3.98,,2018-07-12,148.756000,24.70,3.86
ABC,0.300708,0.514273,0.092449,93.230,1.745962,102.87,1.18,50.0,1.52,3.86,,2018-05-18,20.492000,24.16,1.60


In [ ]:
start_time = time.time()

combos = back_ratio_opt(tickers[tickers['Market Cap (B)'] <= 5].sort_values(['daily_ann','Market Cap (B)']).index.tolist(),
                      10, 1, 2, 'market', 0, 0.05, 30, 210)

print("--- %s seconds ---" % (time.time() - start_time))

In [144]:
interest = unusual_names[(unusual_names['DTE'] >= 90) & 
                         (unusual_names['DTE'] <= 300) &
                         (unusual_names['Open Interest'] >= 1500)].join(tickers, 
                                                             how = 'inner').sort_values(['DTE',
                                                                                         'Market Cap (B)'],
                                                                                        ascending = False)
interest[['Price', 'Type', 'Strike', 'DTE', 'Bid', 'Midpoint', 'Ask',
 'Last', 'Volume', 'Open Interest', 'IV', 
 'daily_ann', 'intra_ann', 'daily_dollar_vol',
 'Beta','Market Cap (B)', 'PE Ratio (TTM)',
 'Yield']].sort_values(['intra_ann'], ascending = False)

,Price,Type,Strike,DTE,Bid,Midpoint,Ask,Last,Volume,Open Interest,IV,daily_ann,intra_ann,daily_dollar_vol,Beta,Market Cap (B),PE Ratio (TTM),Yield
MUSA,80.20,Put,70.0,120,1.45,1.63,1.80,2.10,2371,2278,31.52%,0.333525,0.667365,1.624922,0.91,2.665000,9.88,NaN
MU,59.44,Call,60.0,91,4.95,5.00,5.05,4.85,2762,2545,43.63%,0.372338,0.660107,1.389713,1.55,68.368000,7.11,NaN
MU,59.44,Call,65.0,91,3.00,3.08,3.15,3.04,5070,4663,43.16%,0.372338,0.660107,1.389713,1.55,68.368000,7.11,NaN
DISH,33.75,Call,47.5,92,0.25,0.33,0.40,0.40,2130,1513,47.60%,0.334161,0.602901,0.715916,0.91,16.270000,8.69,NaN
CMCSA,32.79,Call,37.5,147,0.69,0.73,0.77,0.73,5012,2357,25.97%,0.287058,0.552395,0.593302,1.27,153.649000,6.82,2.24
GDEN,29.61,Call,30.0,91,2.70,2.78,2.85,2.80,2761,1932,49.05%,0.279597,0.547427,0.544241,1.02,0.810125,954.19,NaN
VEEV,83.56,Call,90.0,182,5.40,5.60,5.80,5.10,1889,1821,33.45%,0.216371,0.410571,1.133612,1.26,12.125000,91.88,NaN
GM,41.12,Call,46.0,91,0.45,0.47,0.49,0.63,3729,2985,23.17%,0.493623,0.392979,1.314089,1.59,59.126000,N/A,3.49
CAT,139.53,Put,130.0,91,4.05,4.10,4.15,4.20,4115,1714,30.63%,0.225623,0.351970,2.036709,1.50,85.578000,38.68,2.29
CAT,139.53,Call,150.0,91,3.60,3.65,3.70,3.76,3001,2188,26.27%,0.225623,0.351970,2.036709,1.50,85.578000,38.68,2.29


In [102]:
'''
AGO HLT GDEN DISH

'''

In [138]:
combos = test[(abs(test['Delta']) <= 0.01) &
              (abs(test['Long Strike'] - test['Short Strike']) <= 5)]
combos.index = combos['Ticker']
combos.join(tickers)[['Delta', 'Gamma', 'Long Ask', 'Long Bid', 'Long Strike', 'Max Loss',
       'Net Credit', 'Short Ask', 'Short Bid', 'Short Strike', 'Theta', 'Vega',
       'DTE', 'Type', 'Ticker', 'daily_ann', 'intra_ann', 'close',
       'Market Cap (B)', 'PE Ratio (TTM)']]

,Delta,Gamma,Long Ask,Long Bid,Long Strike,Max Loss,Net Credit,Short Ask,Short Bid,Short Strike,Theta,Vega,DTE,Type,Ticker,daily_ann,intra_ann,close,Market Cap (B),PE Ratio (TTM)
AAXN,0.008060,-0.000164,0.25,0.00,15.0,-3.00,0.00,0.65,0.50,18.0,0.002422,-0.007222,210,put,AAXN,0.396124,0.582264,68.45,3.935,271.63
CRC,0.002621,0.001321,0.65,0.35,8.0,1.30,3.30,5.00,4.60,10.0,0.008398,0.002055,210,put,CRC,0.739159,1.187881,40.02,1.930,N/A
CRC,-0.007635,0.006658,1.00,0.70,20.0,0.80,1.80,4.20,3.80,21.0,0.008854,0.011912,119,put,CRC,0.739159,1.187881,40.02,1.930,N/A
CRC,-0.008312,0.004100,0.90,0.65,17.0,-0.05,0.95,3.10,2.75,18.0,0.003666,0.010943,147,put,CRC,0.739159,1.187881,40.02,1.930,N/A
CRC,0.002456,0.003801,0.90,0.65,17.0,-0.60,1.40,3.60,3.20,19.0,0.005670,0.007623,147,put,CRC,0.739159,1.187881,40.02,1.930,N/A
CRC,0.002686,0.002004,0.55,0.30,15.0,-0.70,0.30,1.60,1.40,16.0,0.003029,0.003382,119,put,CRC,0.739159,1.187881,40.02,1.930,N/A
CRC,0.004201,0.003021,0.85,0.55,16.0,-0.95,1.05,3.10,2.75,18.0,0.004827,0.005901,147,put,CRC,0.739159,1.187881,40.02,1.930,N/A
IDTI,-0.004154,0.009711,0.35,0.15,23.0,-0.70,0.30,1.10,1.00,24.0,0.000470,0.010635,147,put,IDTI,0.183987,0.319674,34.53,4.464,N/A
IDTI,0.008046,0.027915,0.75,0.60,27.0,-1.30,1.70,3.60,3.20,30.0,0.002966,0.024000,147,put,IDTI,0.183987,0.319674,34.53,4.464,N/A
MTZ,0.002802,0.004903,0.35,0.15,36.0,-0.70,0.30,1.20,1.00,37.0,0.001491,0.008605,119,put,MTZ,0.273115,0.502304,54.50,4.293,13.49
